In [26]:
# import libraries
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import scipy.integrate as integrate
import scipy.special as special
import math
from matplotlib.colors import ListedColormap

In [27]:
def Tb(rb = 1, Mb = 1e12):
    k = 5.67e-8 # stefan boltzmann constant in W/(m^2*K)
    mmp = 1 # mean mass per particle
    G = 6.67e-11 #m^3/kg/s^2
    vc = (G*Mb/rb)**(1/2)
    T = mmp*vc**2/(2*k)
    return T

In [28]:
def T_phi_iso(r, rb = 1, **kwargs):
    return r/r * Tb(rb)

In [29]:
#define new function
# other functions include T_phi_iso
def T_phi_nfw(r,rb = 1, c = 10, **kwargs):
    rs = rb/c # c is dimensionless
    x = c*r
    M_nfw_b = math.log(1+c) - c/(1+c)
    M_nfw = np.array(x)
    for i in range(0,len(x)):
        M_nfw[i] = math.log(1+x[i])-x[i]/(1+x[i])
    # f_Tx = math.log(1+x)/x - 1/(1+x) which is basically M_nfw/x
    # f_Tc = math.log(1+c)/c - 1/(1+c)
    # f_T is prop to M_nfw/r is prop to Tphi
    # T_phi = T_phi(rb)*f_Tx/f_Tc
    return (M_nfw/r)/(M_nfw_b/rb) * Tb(rb)

In [30]:
def T_phi_mass(r, rb = 1, Mb = 1e12): # unit management
    G = 6.67e-11 #m^3/kg/s^2
    k = 5.67e-8 # stefan boltzmann constant in W/(m^2*K)
    H0 = 70 #km/s/Mpc
    Hz = H0 # could be a parameter
    mmp = 1 # mean mass per particle
    Mb = 4*np.pi*rb**3/3*200*3*Hz**2/(8*np.pi*G) # rearange for rb
    Tphi = G*Mb*mmp/(2*k*r) # k is boltzmann's constant, rb?
    # for milky way solar masses, temp should be 10^6K
    return Tphi

In [31]:
from ipywidgets import interact,interact_manual

def integral_function(K0 = 0.0, pwr = 4/3, T_phi = T_phi_nfw, **kwargs):
    # constants
    n = 1000 # number of intervals
    rb = 250 #kpc
    rmin = 0.01*rb
    rmax = rb
    Krb = 1 #K0 + K1
    K1 = Krb - K0

    ln_rmin = math.log(rmin)
    ln_r = np.zeros((n))

    for i in range(0, n):
        ln_r[i] = ln_rmin+i/(n-1)*math.log(rmax/rmin)
    ln_r
    r = np.exp(ln_r)

    dln_r = math.log(rmax/rmin)/(n-1)
    K = K1*(r/rb)**pwr + K0 # K(r)
    
    # function for Tphi - define Tphi as code function outside
    Tphi = T_phi(r)

        
    d_dlnr = -4/5*Tphi/(K**(3/5)) 

    # initialize arrays
    Ti_Ki = np.zeros((n))
    
    
    Ti_Ki[-1] =  4/(3*pwr)*Tphi[-1] # Tb/Krb**(3/5)
    print(Ti_Ki[-1])
    for j in range(1,n):
        i = n-j


        Ti_Ki[i-1] = Ti_Ki[i] - d_dlnr[i]*dln_r

    Tr = Ti_Ki*(K**(3/5))

    sns.set()

    cmap1 = ListedColormap(sns.color_palette("ch:s=.25,rot=-.25"))
    cmap2 = ListedColormap(sns.color_palette("ch:s=-.2,r=.6"))
    
    colors = np.linspace(Tr[0],Tr[-1],len(Tr))
    plt.scatter(ln_r,Tr, c = colors, cmap=cmap1)
    plt.scatter(ln_r,Tphi, c = colors, cmap=cmap2)
    
    plt.colorbar()
    #plt.show()
    plt.xlabel('ln_r')

    plt.ylabel('Tr')
    
    # I DID IT IM PROUD
    
    #if np.any(Tr) < 2 :
    #    plt.ylim((0,2)) 
        

    sns.set_context('notebook')

In [32]:
interact(integral_function, K0=(0.0,1, .01), pwr=(0.01,2,0.01), T_phi = [("NFW", T_phi_nfw), ("ISO", T_phi_iso), ("MASS", T_phi_mass)]);

interactive(children=(FloatSlider(value=0.0, description='K0', max=1.0, step=0.01), FloatSlider(value=1.333333…